In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData
from MaaSSim.day_to_day import S_driver_opt_out_TS, S_traveller_opt_out_TS
from MaaSSim.day_to_day import d2d_kpi_veh, d2d_kpi_pax

In [4]:
import osmnx as ox
import networkx as nx
import random
import math
import pandas as pd
import matplotlib.pyplot as plt          
import numpy as np
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

In [5]:
params = MaaSSim.utils.get_config('glance.json')
params.city = "Amsterdam, Netherlands"
params.paths.G = "../../data/graphs/Amsterdam.graphml"
params.paths.skim = "../../data/graphs/Amsterdam.csv"
params.vehicle_fixed_positions = True
params.times.patience = 600 #second
params.dist_threshold = 20000 #meter
params.dist_threshold_min = 3000 #meter

params.d2d.probabilistic = True
params.d2d.heterogeneous = False
params.d2d.B_income = 1
params.d2d.fuel_cost = 0.25 #euro/km

params.d2d.B_inveh_time = 1
params.d2d.B_exp_time = 2 #1.5 in oded's paper
params.PT_fare = 1 ## euro
params.PT_avg_speed = 6

params.nPM = 2 # number of platforms. platform id starts from 1
params.platforms.base_fare = 1.4 #euro
params.platforms.fare = float(1.20) #euro/km
params.platforms.min_fare = float(2) # euro
params.platforms.comm_rate = float(0.20)
params.platforms.discount = float(0.0)
params.platforms.daily_marketing = True

params.d2d.learning_d = 1 
params.d2d.B_Experience = 0.80
params.d2d.B_WOM = 0.20
params.d2d.B_Marketing = 0.0
params.d2d.diffusion_speed = 10/100 # speed for M is twice greater than WOM.
params.d2d.m = 5
params.d2d.mn = 100/12 # for NL
params.d2d.ini_att = 0.01
params.d2d.Eini_att = 0.01
params.d2d.adj_s = 2
params.VoT = 10.63 # value of time per hour
params.assert_me = False
params.seed = 1
params.paths.requests = 'Amsterdam_requests.csv'
params.simTime = 4
params.d2d.res_wage = params.simTime*10.63 #euro

In [9]:
params.threshold_u = 0.005
params.max_revenue = 4000 # maximum revenue with the initial fare
params.alpha = 0.5
params.initial_capital = 50000
params.expense_per_day = 500
params.random_ini_position = False
params.step_size = 0.2 # euro/km
params.min_fare, params.max_fare  = 0.0, 3.0
params.initial_fares = (1.0,1.0)
params.min_wage_sub = False

params.nD =2 # 1000
params.turnover_interval =2# 50 #---------------------------------
params.nP = 2000
params.nV = 200

In [10]:
sim = MaaSSim.simulators.Try_and_Select(params=params, f_driver_out=S_driver_opt_out_TS,f_trav_out=S_traveller_opt_out_TS,
                                  kpi_veh=d2d_kpi_veh,kpi_pax=d2d_kpi_pax, logger_level=logging.WARNING)

This simulation uses albatros data
30-12-24 09:53:17-WARNING-Setting up 4h simulation at 2024-12-30 07:52:45 for 200 vehicles and 2000 passengers in Amsterdam, Netherlands
============ P1 TURN ============
30-12-24 09:53:27-WARNING-simulation time 3.6 s
Day =  0 --- on the Left cell with fare 0.8
(np1,vp1) =  (0, 0)     (np2,vp2) =  (0, 0)
--------------------------------------------
30-12-24 09:53:37-WARNING-simulation time 3.9 s
Day =  1 --- on the Left cell with fare 0.8
(np1,vp1) =  (23, 4)     (np2,vp2) =  (18, 1)
--------------------------------------------
--------------------------------------------
P1:  (0.8, 0.01123234, 0.0067146800000000015, 0.01575)
30-12-24 09:53:47-WARNING-simulation time 3.0 s
Day =  0 --- on the Middle cell with fare 1.0
(np1,vp1) =  (0, 0)     (np2,vp2) =  (0, 0)
--------------------------------------------
30-12-24 09:53:57-WARNING-simulation time 3.9 s
Day =  1 --- on the Middle cell with fare 1.0
(np1,vp1) =  (15, 0)     (np2,vp2) =  (21, 2)
-------

In [11]:
sim.res[1].plat_exp

,fare,revenue,profit,nP,nV,min_wage_sub,remaining_capital,P_market_share,V_market_share,market_share
platform_id,,,,,,,,,,
1,0.8,26.85872,-473.14128,23,4,0.0,49026.85872,0.0115,0.02,0.01575
2,1.0,14.262,-485.738,18,1,0.0,49014.262,0.009,0.005,0.007


In [9]:
sim.competition_trajectory['P1']

[(0.8, 0.02625733600000001, 0.02625733600000001, 0.04885),
 (1.0, 0.031562259999999995, 0.031562259999999995, 0.046250000000000006),
 (1.2, 0.033957384, 0.033957384, 0.046575),
 (0.8, 0.03432186400000001, 0.03432186400000001, 0.0641),
 (1.0, 0.037157245000000005, 0.037157245000000005, 0.056950000000000014),
 (1.2, 0.045494693999999995, 0.045494693999999995, 0.059125),
 (1.0, 0.052400675, 0.052400675, 0.074575),
 (1.2, 0.05806447599999999, 0.05806447599999999, 0.074625),
 (1.4, 0.066303069, 0.066303069, 0.08380000000000001),
 (1.2, 0.06902750600000002, 0.06902750600000002, 0.09159999999999999),
 (1.4, 0.07385873600000001, 0.07385873600000001, 0.09259999999999999),
 (1.6, 0.07840420000000002, 0.07840420000000002, 0.097075),
 (1.2, 0.07510809000000002, 0.07510809000000002, 0.09582500000000001),
 (1.4, 0.08292458999999999, 0.08292458999999999, 0.10294999999999999),
 (1.6, 0.09044964000000001, 0.09044964000000001, 0.11112499999999999),
 (1.4, 0.087050019, 0.087050019, 0.10824999999999999),


In [8]:
params.nD = len(sim.res)

df_s = pd.DataFrame()
df_d = pd.DataFrame()

for d in range(0,params.nD):
    
    veh_exp = sim.res[d].veh_exp
    pax_exp = sim.res[d].pax_exp
    plat_exp = sim.res[d].plat_exp
    platforms = sim.res[d].platforms
    
    df_d.at[d, 'P1_EXPERIENCE_U'] = pax_exp.P1_EXPERIENCE_U.mean()
    df_d.at[d, 'P2_EXPERIENCE_U'] = pax_exp.P2_EXPERIENCE_U.mean()
    df_d.at[d, 'P1_WOM_U'] = pax_exp.P1_WOM_U.mean()
    df_d.at[d, 'P2_WOM_U'] = pax_exp.P2_WOM_U.mean()
    df_d.at[d, 'P1_MARKETING_U'] = pax_exp.P1_MARKETING_U.mean()
    df_d.at[d, 'P2_MARKETING_U'] = pax_exp.P2_MARKETING_U.mean()
    df_d.at[d, 'P1_OUT'] = pax_exp[pax_exp.platform_id==1].OUT.value_counts().get(False, 0)
    df_d.at[d, 'P2_OUT'] = pax_exp[pax_exp.platform_id==2].OUT.value_counts().get(False, 0)
    df_d.at[d, 'P1_ACTUAL_WT'] = pax_exp[pax_exp.platform_id==1].ACTUAL_WT.mean()
    df_d.at[d, 'P2_ACTUAL_WT'] = pax_exp[pax_exp.platform_id==2].ACTUAL_WT.mean()
    df_d.at[d, 'P1_MATCHING_T'] = pax_exp[pax_exp.platform_id==1].MATCHING_T.mean()
    df_d.at[d, 'P2_MATCHING_T'] = pax_exp[pax_exp.platform_id==2].MATCHING_T.mean()
    df_d.at[d, 'P1_PICKUP_T'] = veh_exp[veh_exp.platform_id==1].ARRIVES_AT_PICKUP.mean()/(veh_exp[veh_exp.platform_id==1].nRIDES.mean()*60)
    df_d.at[d, 'P2_PICKUP_T'] = veh_exp[veh_exp.platform_id==2].ARRIVES_AT_PICKUP.mean()/(veh_exp[veh_exp.platform_id==2].nRIDES.mean()*60)
    df_d.at[d, 'PICKUP_T'] = veh_exp[veh_exp.OUT==False].ARRIVES_AT_PICKUP.mean()/(veh_exp[veh_exp.OUT==False].nRIDES.mean()*60)
    
    df_d.at[d, 'P1_fare'] = plat_exp.fare[1]; df_d.at[d, 'P2_fare'] = plat_exp.fare[2]
    df_d.at[d, 'P1_revenue'] = plat_exp.revenue[1]; df_d.at[d, 'P2_revenue'] = plat_exp.revenue[2]
    df_d.at[d, 'P1_remaining_capital'] = plat_exp.remaining_capital[1]; df_d.at[d, 'P2_remaining_capital'] = plat_exp.remaining_capital[2]
    
    df_s.at[d, 'P1_EXPERIENCE_U'] = veh_exp.P1_EXPERIENCE_U.mean()
    df_s.at[d, 'P2_EXPERIENCE_U'] = veh_exp.P2_EXPERIENCE_U.mean()
    df_s.at[d, 'P1_WOM_U'] = veh_exp.P1_WOM_U.mean()
    df_s.at[d, 'P2_WOM_U'] = veh_exp.P2_WOM_U.mean()
    df_s.at[d, 'P1_MARKETING_U'] = veh_exp.P1_MARKETING_U.mean()
    df_s.at[d, 'P2_MARKETING_U'] = veh_exp.P2_MARKETING_U.mean()
    df_s.at[d, 'P1_OUT'] = veh_exp[veh_exp.platform_id==1].OUT.value_counts().get(False, 0)
    df_s.at[d, 'P2_OUT'] = veh_exp[veh_exp.platform_id==2].OUT.value_counts().get(False, 0)
    df_s.at[d, 'P1_IDLE_TIME'] = veh_exp[veh_exp.platform_id==1].IDLE_TIME.mean()/60
    df_s.at[d, 'P2_IDLE_TIME'] = veh_exp[veh_exp.platform_id==2].IDLE_TIME.mean()/60
    df_s.at[d, 'P1_REVENUE'] = veh_exp[veh_exp.platform_id==1].REVENUE.mean()
    df_s.at[d, 'P2_REVENUE'] = veh_exp[veh_exp.platform_id==2].REVENUE.mean()
    df_s.at[d, 'P1_ACTUAL_INC'] = veh_exp[veh_exp.platform_id==1].ACTUAL_INC.mean()
    df_s.at[d, 'P2_ACTUAL_INC'] = veh_exp[veh_exp.platform_id==2].ACTUAL_INC.mean()
    df_s.at[d, 'P1_COST'] = veh_exp[veh_exp.platform_id==1].COST.mean()
    df_s.at[d, 'P2_COST'] = veh_exp[veh_exp.platform_id==2].COST.mean()
    
    if d==0: d_cols = df_d.columns.values.tolist()
    if d==0: s_cols = df_s.columns.values.tolist()
    
    ld = sim.res[0].pax_exp.columns.values.tolist()
    ldn = [i for i in ld if i not in d_cols]
    for col in ldn:
        df_d.at[d, col] = pax_exp[pax_exp.OUT==False][col].mean()
        
    ls = sim.res[0].veh_exp.columns.values.tolist()
    lsn = [i for i in ls if i not in s_cols]
    for col in lsn:
        df_s.at[d, col] = veh_exp[veh_exp.OUT==False][col].mean()
        
run_id = '{}_1_1'.format(params.alpha)
df_d.to_csv('csv/1.1/demand_{}.csv'.format(run_id))
df_s.to_csv('csv/1.1/supply_{}.csv'.format(run_id))

In [11]:
params.alpha = 0.6

simx = MaaSSim.simulators.Try_and_Select(params=params, f_driver_out=S_driver_opt_out_TS,f_trav_out=S_traveller_opt_out_TS,
                                  kpi_veh=d2d_kpi_veh,kpi_pax=d2d_kpi_pax, logger_level=logging.WARNING)

This simulation uses albatros data
27-12-24 01:21:03-WARNING-Setting up 4h simulation at 2024-12-26 23:20:31 for 200 vehicles and 2000 passengers in Amsterdam, Netherlands
============ P1 TURN ============
27-12-24 01:21:12-WARNING-simulation time 3.0 s
Day =  0 --- on the Left cell with fare 0.8
(np1,vp1) =  (0, 0)     (np2,vp2) =  (0, 0)
--------------------------------------------
27-12-24 01:21:23-WARNING-simulation time 4.0 s
Day =  1 --- on the Left cell with fare 0.8
(np1,vp1) =  (12, 1)     (np2,vp2) =  (20, 4)
--------------------------------------------
27-12-24 01:21:38-WARNING-simulation time 4.4 s
Day =  2 --- on the Left cell with fare 0.8
(np1,vp1) =  (31, 3)     (np2,vp2) =  (30, 2)
--------------------------------------------
27-12-24 01:21:50-WARNING-simulation time 5.0 s
Day =  3 --- on the Left cell with fare 0.8
(np1,vp1) =  (38, 5)     (np2,vp2) =  (47, 2)
--------------------------------------------
27-12-24 01:22:03-WARNING-simulation time 5.9 s
Day =  4 --- on 